In [1]:
!pip install rdkit
!pip install pysmiles
!pip install networkx
!pip install spektral

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
pp = pd.read_csv('../data/test.csv')
pp = pp[["name", "p_np", "smile"]]
pp.head()

,name,p_np,smile
0,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


In [3]:
from pysmiles import read_smiles
import networkx as nx
from spektral.data import Dataset, DisjointLoader, Graph
import numpy as np
import random

2023-12-04 01:55:55.736586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
tempList = []

##hashing here sucks, should find a better way

for index, row in pp.iterrows():
    smiles = row["smile"]
    mol = read_smiles(smiles)
    nx.convert_node_labels_to_integers(mol)
    nodevec = list(mol.nodes(data='element'))

    ##change this later to the actual element value, just tryna get some loss and the nn actually working
    nodevec = [1 for x in nodevec]
    adj = nx.to_pandas_adjacency(mol).to_numpy().astype(int)


    ##change this later to the actual element value, just tryna get some loss and the nn actually working
    g = Graph(x = np.array(nodevec), a = adj, y = random.randint(1, 100))
    tempList.append(g)


/Users/markyukelis/Library/Python/3.8/lib/python/site-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (20, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")
/Users/markyukelis/Library/Python/3.8/lib/python/site-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (23, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")
/Users/markyukelis/Library/Python/3.8/lib/python/site-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (26, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")
/Users/markyukelis/Library/Python/3.8/lib/python/site-packages/spektral/data/graph.py:67: UserWarning: x was automatically reshaped to (21, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@H]" contains stereochemical information that will be discarded.
Atom "[C@@H]" contains stereochemi

In [21]:
pp["graph"] = tempList

graphs = pp["graph"]
graphs


0       Graph(n_nodes=20, n_node_features=1, n_edge_fe...
1       Graph(n_nodes=23, n_node_features=1, n_edge_fe...
2       Graph(n_nodes=26, n_node_features=1, n_edge_fe...
3       Graph(n_nodes=21, n_node_features=1, n_edge_fe...
4       Graph(n_nodes=29, n_node_features=1, n_edge_fe...
                              ...                        
2032    Graph(n_nodes=17, n_node_features=1, n_edge_fe...
2033    Graph(n_nodes=27, n_node_features=1, n_edge_fe...
2034    Graph(n_nodes=24, n_node_features=1, n_edge_fe...
2035    Graph(n_nodes=28, n_node_features=1, n_edge_fe...
2036    Graph(n_nodes=21, n_node_features=1, n_edge_fe...
Name: graph, Length: 2037, dtype: object

In [22]:
class MyDataset(Dataset):
    """
    load our data
    """
    def __init__(self, graphs, **kwargs):
        self.graphs = graphs
        super().__init__(**kwargs)
    

        
    def read(self):
        # We must return a list of Graph objects
        return self.graphs


In [23]:
dataset = MyDataset(graphs = graphs)
dataset

MyDataset(n_graphs=2037)

In [24]:
dataset.filter(lambda g: g.n_nodes < 500)
dataset

MyDataset(n_graphs=2037)

In [25]:
max_degree = dataset.map(lambda g: g.a.sum(-1).max(), reduce=max)
max_degree

5

In [26]:
from spektral.transforms import Degree

dataset.apply(Degree(max_degree))
dataset[0]

Graph(n_nodes=20, n_node_features=7, n_edge_features=None, n_labels=1)

In [27]:
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out


In [29]:
model = MyFirstGNN(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

In [36]:
from spektral.data import BatchLoader

np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
data_tr, data_te = dataset[:split], dataset[split:]

loader = BatchLoader(data_tr, batch_size=32)

/var/folders/g5/5hm313zd2m5gg0k6hkp7bt_c0000gn/T/ipykernel_18394/873634478.py:3: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


In [37]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
38/51 [=====================>........] - ETA: 0s - loss: 0.0000e+00

/Users/markyukelis/Library/Python/3.8/lib/python/site-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'MyDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 2/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 3/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 4/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 5/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 6/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 7/10
51/51 [==============================] - 0s 4ms/step - loss: 0.0000e+00
Epoch 8/10
51/51 [==============================] - 0s 5ms/step - loss: 0.0000e+00
Epoch 9/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Epoch 10/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0000e+00


In [38]:
loader = BatchLoader(data_te, batch_size=32)
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))


13/13 [==============================] - 0s 3ms/step - loss: 0.0000e+00
Test loss: 0.0


In [43]:
data_te[0]
data_te[0].x

array([[1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0.]])